## **silver-companies-income-statement**
<p>This notebook ingests unprocessed income statement data such as revenue, gross profit, net income, and EPS. The data forms a foundational layer for further financial modeling and company performance tracking.</p>

#### **Importing necessary libraries**

In [1]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, e65341ec-388e-48ba-bed8-214f4998ae91, 5, Finished, Available, Finished)

#### **Variables**

In [2]:
file_name = "comp_incm_stmt_data"
layer_name = "bronze"

StatementMeta(, e65341ec-388e-48ba-bed8-214f4998ae91, 6, Finished, Available, Finished)

#### **Step 2: Read the Balance Sheet and Breakdown Categories Data**

In [3]:
incm_df = spark.read.parquet(f"Files/{layer_name}/{file_name}/")

breakdown_categories = spark.read.csv("Files/master_files/breakdown_categories.csv", header=True)

StatementMeta(, e65341ec-388e-48ba-bed8-214f4998ae91, 7, Finished, Available, Finished)

#### **Step 3: Filters the Unnecessary Breakdown Categories from the Balance Sheet**

In [4]:
output_df = incm_df\
                    .join(broadcast(breakdown_categories), incm_df.Breakdown == breakdown_categories['Account Name'], 'inner')\
                    .select(incm_df.Breakdown, incm_df.Company, incm_df.Value.alias('amount'), incm_df.Date)

StatementMeta(, e65341ec-388e-48ba-bed8-214f4998ae91, 8, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Silver layer**

In [5]:
delta_path = f"Files/silver/{file_name}"

try:
    if DeltaTable.isDeltaTable(spark, delta_path):
        delta_table = DeltaTable.forPath(spark, delta_path)
        
        delta_table.alias("target").merge(
            output_df.alias("source"),
                    (col("target.Breakdown") == col("source.Breakdown")) &
                    (col("target.Company") == col("source.Company")) &
                    (col("target.Date") == col("source.Date"))
        ).whenMatchedUpdateAll(
        ).whenNotMatchedInsertAll().execute()

    else:
        output_df.write.format("delta").mode("overwrite").save(delta_path)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, e65341ec-388e-48ba-bed8-214f4998ae91, 9, Finished, Available, Finished)